# import packages

In [24]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split



# read data

In [2]:
# read DATA
file_path = "../data/filled_data.xlsx"
filled_data = pd.read_excel(file_path,engine='openpyxl')
filled_data

,serial_no,gender,dept_flg,area_flg,cntr_flg,agent_level,agent_status,fyp_month_avg,persistence_prem_25,salary_ded_ratio,...,rule30_counts_missing_flag,rule31_counts_missing_flag,rule32_counts_missing_flag,rule33_counts_missing_flag,rule34_counts_missing_flag,rule35_counts_missing_flag,rule36_counts_missing_flag,rule37_counts_missing_flag,rule38_counts_missing_flag,rule39_counts_missing_flag
0,agnt_0001,M,DEPT3,AREA13,CNTR13,K7,3,30054.25,94.42,33.33,...,0,0,0,0,0,0,0,0,0,0
1,agnt_0002,M,DEPT2,AREA18,CNTR12,HA,1,68389.32,100.00,33.33,...,0,0,0,0,0,0,0,0,0,0
2,agnt_0003,M,DEPT3,AREA13,CNTR13,Z1,2,14618.46,94.42,33.33,...,0,0,0,0,0,0,0,0,0,0
3,agnt_0004,M,DEPT3,AREA13,CNTR35,Z1,2,42400.30,94.42,33.33,...,0,0,0,0,0,0,0,0,0,0
4,agnt_0005,M,DEPT3,AREA13,CNTR35,Z1,1,23869.50,0.00,33.33,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4014,agnt_4015,F,DEPT3,AREA13,CNTR13,Z1,2,30268.00,94.42,33.33,...,0,0,0,0,0,0,0,0,0,0
4015,agnt_4016,F,DEPT3,AREA14,CNTR17,Z1,2,44524.35,94.42,33.33,...,0,0,0,0,0,0,0,0,0,0
4016,agnt_4017,M,DEPT3,AREA13,CNTR35,KA,2,36144.52,94.42,33.33,...,0,0,0,0,0,0,0,0,0,0
4017,agnt_4018,F,DEPT2,AREA21,CNTR14,Z1,2,30268.00,94.42,33.33,...,0,0,0,0,0,0,0,0,0,0


# 未過採樣

## preprocessing : 把類別型轉成數字

In [32]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# 讀取資料
file_path = r"G:\我的雲端硬碟\NCCU\Machine Learning and AI Case Implementation\NCCU_ML_AI\data\filled_data.xlsx"
filled_data_df = pd.read_excel(file_path, engine="openpyxl")

# 目標欄位
target_col = "abnormal_target"

# 確認類別型欄位
categorical_columns = [
    "gender", "dept_flg", "area_flg", "cntr_flg", "agent_level", "agent_status", 
    "xagfd_flag", "xagdt_flag", "cusp_own_flag", "cusp_intr_flag"
]

# 刪除不必要的欄位，如 serial_no
filled_data_df = filled_data_df.drop(columns=["serial_no"])

# 將所有類別型欄位的值轉換為字串
for col in categorical_columns:
    filled_data_df[col] = filled_data_df[col].astype(str)

# 對類別型欄位進行 Label Encoding
label_encoder = LabelEncoder()

for col in categorical_columns:
    filled_data_df[col] = label_encoder.fit_transform(filled_data_df[col])

# 儲存每個欄位的映射結果
mapping_dict = {}

for col in categorical_columns:
    label_encoder.fit(filled_data_df[col])  # 為每一列進行 fit
    mapping_dict[col] = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))

# 創建 DataFrame 以便更容易查看
mapping_df = pd.DataFrame(mapping_dict)

# 顯示清晰的映射表格
mapping_df




,gender,dept_flg,area_flg,cntr_flg,agent_level,agent_status,xagfd_flag,xagdt_flag,cusp_own_flag,cusp_intr_flag
0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,1,1,1.0,1.0,1.0,1.0,1.0
2,NaN,2.0,2.0,2,2,2.0,NaN,NaN,NaN,NaN
3,NaN,3.0,3.0,3,3,3.0,NaN,NaN,NaN,NaN
4,NaN,4.0,4.0,4,4,4.0,NaN,NaN,NaN,NaN
5,NaN,NaN,5.0,5,5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,6.0,6,6,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,7.0,7,7,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,8.0,8,8,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,9.0,9,9,NaN,NaN,NaN,NaN,NaN


In [33]:
filled_data_df

,gender,dept_flg,area_flg,cntr_flg,agent_level,agent_status,fyp_month_avg,persistence_prem_25,salary_ded_ratio,xagfd_flag,...,rule30_counts_missing_flag,rule31_counts_missing_flag,rule32_counts_missing_flag,rule33_counts_missing_flag,rule34_counts_missing_flag,rule35_counts_missing_flag,rule36_counts_missing_flag,rule37_counts_missing_flag,rule38_counts_missing_flag,rule39_counts_missing_flag
0,1,2,4,4,25,2,30054.25,94.42,33.33,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,9,3,11,0,68389.32,100.00,33.33,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2,4,4,33,1,14618.46,94.42,33.33,0,...,0,0,0,0,0,0,0,0,0,0
3,1,2,4,28,33,1,42400.30,94.42,33.33,0,...,0,0,0,0,0,0,0,0,0,0
4,1,2,4,28,33,0,23869.50,0.00,33.33,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4014,0,2,4,4,33,1,30268.00,94.42,33.33,0,...,0,0,0,0,0,0,0,0,0,0
4015,0,2,5,8,33,1,44524.35,94.42,33.33,0,...,0,0,0,0,0,0,0,0,0,0
4016,1,2,4,28,27,1,36144.52,94.42,33.33,0,...,0,0,0,0,0,0,0,0,0,0
4017,0,1,13,5,33,1,30268.00,94.42,33.33,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
# 儲存處理過的 DataFrame 到 Excel 檔案
output_file_path = r"G:\我的雲端硬碟\NCCU\Machine Learning and AI Case Implementation\NCCU_ML_AI\data\filled_data_label_numeric.xlsx"
filled_data_df.to_excel(output_file_path, index=False, engine="openpyxl")

print(f"檔案已儲存至: {output_file_path}")


檔案已儲存至: G:\我的雲端硬碟\NCCU\Machine Learning and AI Case Implementation\NCCU_ML_AI\data\filled_data_label_numeric.xlsx


## 分割訓練集跟測試集

In [37]:
# 特徵欄位
X = filled_data_df.drop(columns=[target_col])

# 目標變數
y = filled_data_df[target_col]

# 分割數據為訓練集和測試集（80%訓練，20%測試）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"訓練集大小: {X_train.shape[0]}, 測試集大小: {X_test.shape[0]}")

訓練集大小: 3215, 測試集大小: 804


## 訓練模型

In [38]:
# 計算正負樣本比例，並設置 scale_pos_weight
scale_pos_weight = (y_train.value_counts()[0] / y_train.value_counts()[1])

# 設定 XGBoost 的參數
params = {
    'objective': 'binary:logistic',  # 二元分類
    'eval_metric': 'logloss',        # 損失函數
    'max_depth': 6,                  # 樹的最大深度
    'learning_rate': 0.1,            # 學習率
    'n_estimators': 100,             # 迭代次數
    'use_label_encoder': False,      # 禁用 label encoder
    'early_stopping_rounds': 10,     # 早停，防止過度擬合
    'scale_pos_weight': scale_pos_weight  # 設置權重來應對不平衡資料
}

# 創建驗證集，使用 20% 測試集作為驗證集
eval_set = [(X_test, y_test)]

# 建立模型
model = xgb.XGBClassifier(**params)

# 訓練模型，並提供驗證集進行早停
model.fit(X_train, y_train, eval_set=eval_set, verbose=True)




[0]	validation_0-logloss:0.60101
[1]	validation_0-logloss:0.52592
[2]	validation_0-logloss:0.46282
[3]	validation_0-logloss:0.40975
[4]	validation_0-logloss:0.36458
[5]	validation_0-logloss:0.32564
[6]	validation_0-logloss:0.29173
[7]	validation_0-logloss:0.26229
[8]	validation_0-logloss:0.23584
[9]	validation_0-logloss:0.21317
[10]	validation_0-logloss:0.19267
[11]	validation_0-logloss:0.17510
[12]	validation_0-logloss:0.15917
[13]	validation_0-logloss:0.14525
[14]	validation_0-logloss:0.13315
[15]	validation_0-logloss:0.12197
[16]	validation_0-logloss:0.11235
[17]	validation_0-logloss:0.10365
[18]	validation_0-logloss:0.09551
[19]	validation_0-logloss:0.08878
[20]	validation_0-logloss:0.08230
[21]	validation_0-logloss:0.07677
[22]	validation_0-logloss:0.07143
[23]	validation_0-logloss:0.06675
[24]	validation_0-logloss:0.06248
[25]	validation_0-logloss:0.05868
[26]	validation_0-logloss:0.05522
[27]	validation_0-logloss:0.05209
[28]	validation_0-logloss:0.04930
[29]	validation_0-loglos

c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\callback.py:386: UserWarning: [04:56:02] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  self.starting_round = model.num_boosted_rounds()


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=10,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

## 模型評分

In [39]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 預測測試集
y_pred = model.predict(X_test)

# 計算準確率
accuracy = accuracy_score(y_test, y_pred)
print(f"準確率: {accuracy:.4f}")

# 生成分類報告
print("分類報告:")
print(classification_report(y_test, y_pred))

# 混淆矩陣
conf_matrix = confusion_matrix(y_test, y_pred)
print("混淆矩陣:")
print(conf_matrix)


準確率: 0.9950
分類報告:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       795
           1       0.86      0.67      0.75         9

    accuracy                           1.00       804
   macro avg       0.93      0.83      0.87       804
weighted avg       0.99      1.00      0.99       804

混淆矩陣:
[[794   1]
 [  3   6]]


# 嘗試過採樣

## 用跟未過採用同個數據

In [40]:
# # 特徵欄位
# X = filled_data_df.drop(columns=[target_col])

# # 目標變數
# y = filled_data_df[target_col]

# # 分割數據為訓練集和測試集（80%訓練，20%測試）
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 使用 SMOTE 進行過採樣
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print(f"訓練集原始大小: {X_train.shape[0]}, 過採樣後大小: {X_train_resampled.shape[0]}")


訓練集原始大小: 3215, 過採樣後大小: 6372


## 建模

In [41]:
# 計算過採樣後的正負樣本比例，並設置 scale_pos_weight
scale_pos_weight_resampled = (y_train_resampled.value_counts()[0] / y_train_resampled.value_counts()[1])

# 設定 XGBoost 的參數
params = {
    'objective': 'binary:logistic',  # 二元分類
    'eval_metric': 'logloss',        # 損失函數
    'max_depth': 6,                  # 樹的最大深度
    'learning_rate': 0.1,            # 學習率
    'n_estimators': 100,             # 迭代次數
    'use_label_encoder': False,      # 禁用 label encoder
    'early_stopping_rounds': 10,     # 早停，防止過度擬合
    'scale_pos_weight': scale_pos_weight_resampled,  # 設置權重來應對不平衡資料
}

# 創建驗證集，使用 20% 測試集作為驗證集
eval_set = [(X_test, y_test)]

# 建立模型
model = xgb.XGBClassifier(**params)

# 訓練模型，並提供驗證集進行早停
model.fit(X_train_resampled, y_train_resampled, eval_set=eval_set, verbose=True)


[0]	validation_0-logloss:0.60176
[1]	validation_0-logloss:0.52690
[2]	validation_0-logloss:0.46464
[3]	validation_0-logloss:0.41219
[4]	validation_0-logloss:0.36707
[5]	validation_0-logloss:0.32866
[6]	validation_0-logloss:0.29517
[7]	validation_0-logloss:0.26630
[8]	validation_0-logloss:0.24106
[9]	validation_0-logloss:0.21886
[10]	validation_0-logloss:0.19922
[11]	validation_0-logloss:0.18194
[12]	validation_0-logloss:0.16604
[13]	validation_0-logloss:0.15301
[14]	validation_0-logloss:0.14071
[15]	validation_0-logloss:0.12988
[16]	validation_0-logloss:0.12044
[17]	validation_0-logloss:0.11210
[18]	validation_0-logloss:0.10462
[19]	validation_0-logloss:0.09771
[20]	validation_0-logloss:0.09092
[21]	validation_0-logloss:0.08580
[22]	validation_0-logloss:0.08066
[23]	validation_0-logloss:0.07613
[24]	validation_0-logloss:0.07235
[25]	validation_0-logloss:0.06762
[26]	validation_0-logloss:0.06336
[27]	validation_0-logloss:0.06069
[28]	validation_0-logloss:0.05689
[29]	validation_0-loglos

c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\callback.py:386: UserWarning: [04:56:15] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  self.starting_round = model.num_boosted_rounds()


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=10,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

## 評估模型

In [42]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 用測試集進行預測
y_pred = model.predict(X_test)

# 計算準確率
accuracy = accuracy_score(y_test, y_pred)
print(f"準確率: {accuracy:.4f}")

# 顯示分類報告
print("分類報告:")
print(classification_report(y_test, y_pred))

# 顯示混淆矩陣
print("混淆矩陣:")
print(confusion_matrix(y_test, y_pred))


準確率: 0.9950
分類報告:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       795
           1       0.78      0.78      0.78         9

    accuracy                           1.00       804
   macro avg       0.89      0.89      0.89       804
weighted avg       1.00      1.00      1.00       804

混淆矩陣:
[[793   2]
 [  2   7]]


In [45]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# 設定 XGBoost 的參數範圍
param_dist = {
    'max_depth': [3, 6, 10, 12, 15],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'n_estimators': [50, 100, 150, 200],
    'scale_pos_weight': [scale_pos_weight_resampled],
}

# 設定模型
model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss', use_label_encoder=False)

# 使用 RandomizedSearchCV 尋找最佳參數，這裡使用 'recall' 或 'f1' 作為 scoring
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, cv=3, verbose=2, n_jobs=-1, scoring='recall', random_state=42)

# 訓練模型
random_search.fit(X_train_resampled, y_train_resampled)

# 顯示最佳參數
print("最佳參數:", random_search.best_params_)

# 使用最佳參數的模型預測
best_model_random = random_search.best_estimator_
y_pred_best_random = best_model_random.predict(X_test)

# 計算準確率
accuracy_best_random = accuracy_score(y_test, y_pred_best_random)
print(f"最佳隨機搜尋模型準確率: {accuracy_best_random:.4f}")

# 顯示分類報告
print("最佳隨機搜尋模型分類報告:")
print(classification_report(y_test, y_pred_best_random))

# 顯示混淆矩陣
print("最佳隨機搜尋模型混淆矩陣:")
print(confusion_matrix(y_test, y_pred_best_random))



Fitting 3 folds for each of 10 candidates, totalling 30 fits
最佳參數: {'scale_pos_weight': np.float64(1.0), 'n_estimators': 150, 'max_depth': 3, 'learning_rate': 0.1}
最佳隨機搜尋模型準確率: 0.9925
最佳隨機搜尋模型分類報告:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       795
           1       0.67      0.67      0.67         9

    accuracy                           0.99       804
   macro avg       0.83      0.83      0.83       804
weighted avg       0.99      0.99      0.99       804

最佳隨機搜尋模型混淆矩陣:
[[792   3]
 [  3   6]]


c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [05:10:48] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
